In [ ]:
# !pip install datasets

In [ ]:
# !pip install trl==0.12.2

In [ ]:
# !pip install -U bitsandbytes

In [ ]:
# !pip install -U transformers

In [ ]:
# !pip install langchain_community

In [ ]:
# !pip install -U langchain_experimental

In [ ]:
# !pip install llama-index

In [ ]:
# !pip install llama-index.embeddings.huggingface

In [ ]:
# !pip install llama-index.llms.huggingface

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
import os
import json
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig
from datasets import Dataset
from trl import SFTTrainer

In [ ]:
# QLoRA 모델 로드 순서
# 1. 기본 모델을 4-bit 양자화하여 로드
# 2. LoRA 어댑터 (기존 LoRA 모델) 를 추가하여 QLoRA 모델로 변환

In [ ]:
# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
# 4-bit 양자화된 기본 모델 로드
base_model_name = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'  # Hugging Face 모델 or 로컬 경로
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token  # 패딩 토큰 설정
tokenizer.padding_side = 'right'  # 패딩 방향 설정

# 전체 모델을 GPU로 강제 이동
base_model.to("cuda")

base_model.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [ ]:
# 훈련 하이퍼 파라미터
output_dir = "/content/drive/MyDrive/Colab Notebooks/kollama"

training_args = TrainingArguments(
    per_device_train_batch_size = 2, # 각 GPU당 배치 사이즈 - 기본값은 8
    gradient_accumulation_steps = 1, # 기울기 축적 단계 - 기본값은 1, 커질수록 정확도가 커짐, 곱하기 개념.
                                     # 값을 늘리면 처리 속도는 빨라지지만 GPU 사용량이 커짐
    gradient_checkpointing = True, # 기본값은 False이며 True로 설정하면 역 방향 패스 시 메모리를 절약함
    max_grad_norm = 1.0, # 최대 기울기 표준 - 기본값은 1.0이며 gradient clipping을 위한 변수
                         # Gradient Clipping : Gradient Explosion 문제를 방지하기 위해 기울기의 크기를 특정 임계값 이하로 제한하는 기법
    num_train_epochs = 8, # 훈련할 총 에포크 수
    learning_rate = 1e-5, # 학습률
    bf16 = True, # 32비트 훈련 대신 bf16비트 혼합된 정밀도 훈련을 사용할 지 여부
    save_total_limit = 3, # 설정 시, 오래된 체크포인트 순으로 삭제함
    logging_steps = 10, # 로깅하는 단위 (logging=10 이면 10번마다 로깅)
    output_dir = output_dir, # 결과 경로
    optim = "paged_adamw_32bit", # 사용하려는 옵티마이저 종류 (adam_hf, adamw_torch, adamw_apex_fused, adafactor 등이 있음)
    lr_scheduler_type = "cosine", # 사용하는 스케줄러 타입 - 기본값은 linear, 종류는 cosine 등이 있음
    warmup_ratio = 0.05, # 웜업 비율 - 기본값은 0
    max_steps = -1, # 수행할 총 훈련 단계 수 - 기본값은 -1
                    # overrides_num_train_epoch 설정한 스텝 도달 전에 돌릴 수 있는 모든 데이터가 소진되면 멈춤
                    # 데이터셋 크기가 작기 때문에 너무 많은 스텝을 돌리면 기존 모델의 지식을 과도하게 덮어쓸 수도 있음
    report_to = 'tensorboard', # 결과와 로그를 리포트 저장하는 장소 - all이 기본값이며 all로 설정하는 경우
                               # 연결된 모든 리포트 도구에 저장함 (azure_ml, comet_ml, mlflow, tensorboard, wandb 등이 있음)
    remove_unused_columns = False,  # 중요 입력 데이터 삭제 방지
    weight_decay = 0.05 # 규제
)

In [ ]:
import pandas as pd

#JSON 파일로드
with open(r"/content/drive/MyDrive/Colab Notebooks/scenario/txt/finetuning_dataset2.json", 'r', encoding = "utf-8") as f:
    raw_data = json.load(f)

# 토큰화 함수 정의
def tokenize_function(example):
    # 전체 대화 내용을 하나의 문자열로 결합
    full_conversation = ""
    for message in example['messages']:
        role = message['role']
        content = message['content']
        full_conversation += f"<|{role}|>: {content}\n"  # <|role|>: 내용 형식으로 변환

    # 토크나이즈
    tokenized = tokenizer(
        full_conversation,
        padding='max_length',
        truncation=True,
        max_length=512,
    )

    # labels 생성 (입력과 동일하게)
    tokenized['labels'] = tokenized['input_ids'].copy()

    return tokenized

# 리스트를 Dataset으로 변환
dataset = Dataset.from_list(raw_data)

# 데이터셋에 토큰화 함수 적용
tokenized_dataset = dataset.map(tokenize_function, remove_columns=dataset.column_names)

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [ ]:
# 결과 확인
print(tokenized_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 23
})


In [ ]:
# import pandas as pd

# #JSON 파일로드
# with open(r"/content/drive/MyDrive/Colab Notebooks/scenario/txt/finetuning_dataset.json", 'r', encoding = "utf-8") as f:
#     raw_data = json.load(f)

# # 토큰화 함수 정의
# def tokenize_function(example):
#     # 전체 대화 내용을 하나의 문자열로 결합
#     full_conversation = ""
#     for message in example['messages']:
#         role = message['role']
#         content = message['content']
#         full_conversation += f"<|{role}|>: {content}\n"

#     # 토크나이즈
#     tokenized = tokenizer(
#         full_conversation,
#         padding='max_length',
#         truncation=True,
#         max_length=512,
#     )

#     # labels 생성
#     tokenized['labels'] = tokenized['input_ids'].copy()

#     return tokenized

# # 리스트를 Dataset으로 변환
# dataset = Dataset.from_list(raw_data)

# # 데이터셋에 토큰화 함수 적용
# tokenized_dataset = dataset.map(tokenize_function, remove_columns=dataset.column_names)

In [ ]:
# Lora 어댑터 추가
lora_config = LoraConfig(
    r=4,  # rank
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM" # 인과 언어 모델링 - 문맥 기반 생성, 대화형 생성, 연속적인 텍스트 생성
)

In [ ]:
# 모델에 Lora 어댑터 적용
model = get_peft_model(base_model, lora_config)

In [ ]:
model.enable_input_require_grads()

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    dataset_text_field="input_ids",
    tokenizer = tokenizer,
    max_seq_length = 512,
    args = training_args,
)

trainer.train()
trainer.save_model(output_dir)

output_dir = os.path.join(output_dir, "KoLLaMA_checkpoint")
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set autom

Step,Training Loss
10,2.815400
20,2.844100
30,2.700200
40,2.664100
50,2.612100
60,2.482100
70,2.515900
80,2.429900
90,2.449500


('/content/drive/MyDrive/Colab Notebooks/kollama/KoLLaMA_checkpoint/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/kollama/KoLLaMA_checkpoint/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/kollama/KoLLaMA_checkpoint/tokenizer.json')

In [ ]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [ ]:
# PROMPT = '''당신은 항상 보이스피싱 범죄자 역할을 하는 AI 튜터입니다. 절대 피해자 역할을 하지 않습니다.
#     당신의 역할은 보이스피싱 사기에 사용될 수 있는 다양한 상황 중 하나를 생성하고 상대방과 대화를 나누며, 상대방을 속이려 노력하세요.

#     **보이스피싱 시뮬레이션을 진행하는 방법**
#     - 먼저 **랜덤한 보이스피싱 시나리오**를 하나 선택해서 사용자에게 제시해.
#     - 사용자가 대응할 수 있도록 **실감 나게 연기**해.
#     - 사용자가 대화 종료 라고 입력하면 대화를 종료하고 **올바른 대응인지 피드백을 제공**해.
#     - 피드백을 제공할 때는 상세하게 제공해주고, 사용자의 대응과 별개로 올바른 대응이 어떤건지 알려줘.
#     - 대화가 종료되기 전까지는 보이스피싱 가해자처럼 계속 사용자를 속이려고 노력해야해.
#     - 대화에는 너의 질문과 나의 답변 외에는 출력하지 않도록 해.
#     - 사용자가 올바르게 대응하면 칭찬하고, 부족하면 어떻게 대응해야 하는지 알려줘.
#     - 문장은 대부분 1문장에서 2문장씩만 이야기 해.

#     **보이스피싱 시나리오 예시**
#     - 경찰 사칭: "고객님의 계좌에서 불법 거래가 감지되었습니다."
#     - 은행 사칭: "대출 승인이 완료되었으니 계좌 정보를 입력해주세요."
#     - 대출 사기: "신용등급을 올리려면 보증금이 필요합니다."
#     - 가족 납치: "아드님이 납치되었습니다. 돈을 입금하세요."
#     - 협박: "당신의 개인정보를 해킹했습니다. 돈을 보내지 않으면 유출하겠습니다."

#     **목표: 사용자가 보이스피싱을 잘 구별하고, 올바르게 대응하도록 돕기**'''

# # 대화 이력을 저장하는 리스트. 최초 시스템 메시지만 담아 시작합니다.
# conversation_history = [
#     {"role": "system", "content": PROMPT}
# ]

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# print("대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.\n")

# while True:
#     # 사용자 입력 받기
#     user_input = input("User: ")

#     if user_input.lower() in ["quit", "exit"]:
#         break

#     # 사용자 메시지를 대화 이력에 추가
#     conversation_history.append({"role": "user", "content": user_input})

#     input_ids = tokenizer.apply_chat_template(
#         conversation_history,
#         add_generation_prompt=True,
#         return_tensors="pt"
#     ).to(model.device)

#     # 모델로부터 응답 생성
#     outputs = model.generate(
#         input_ids,
#         max_new_tokens=2048,
#         eos_token_id=terminators,
#         do_sample=True,
#         temperature=0.6,
#         top_p=0.9
#     )

#     # 대화 이력의 길이만큼을 건너뛰고 새로 생성된 텍스트만 추출
#     generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
#     print("Assistant:", generated_text, "\n")

#     # 모델의 응답을 대화 이력에 추가하여, 이후의 대화에 참고되도록 함
#     conversation_history.append({"role": "assistant", "content": generated_text})

***RAG***

In [ ]:
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.llms.huggingface import HuggingFaceLLM
# from llama_index.core.settings import Settings

# # "Private-Data" 폴더 내 PDF 문서 로드
# resume = SimpleDirectoryReader("/content/drive/MyDrive/Colab Notebooks/rag/ragdata").load_data()

# # 트리 인덱스(TreeIndex) 생성
# embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
# new_index =VectorStoreIndex.from_documents(resume, embed_model=embed_model)

# # # Hugging Face 기반 LLM (KoLLaMA)
# # llm = HuggingFaceLLM(model_name='MLP-KTLim/llama-3-Korean-Bllossom-8B')

# # # OpenAI의 GPT 대신 Hugging Face LLM(KoLLaMA) 사용
# # Settings.llm = llm

# # OpenAI API 사용 안 함
# Settings.llm = None

In [ ]:
# # 쿼리 엔진 생성
# query_engine = new_index.as_query_engine()

# response_scenario = query_engine.query("보이스피싱 시뮬레이션에 사용할 내용을 제공해줘.")
# response_strategy = query_engine.query("보이스피싱에 대한 올바른 대응 방법을 설명해줘.")
# print(response_scenario)
# print(response_strategy)

# # 🔥 응답이 나온 문서의 출처 확인
# print(response_scenario.source_nodes)  # 문서 출처를 출력
# print(response_strategy.source_nodes)

In [ ]:
# 문서 없이 벡터를 저장해서 불러올 경우에는 아래 코드 사용

In [ ]:
# new_index.storage_context.persist()

In [ ]:
# from llama_index.core import StorageContext, load_index_from_storage

# 저장된 인덱스를 다시 로드
# storage_context = StorageContext.from_defaults(persist_dir="/content/drive/MyDrive/Colab Notebooks/rag/storage")
# new_index = load_index_from_storage(storage_context)

In [ ]:
# # RAG를 이용해 받은 response.response 값
# method_scenario = response_scenario.response
# method_strategy = response_strategy.response

# PROMPT = '''당신은 항상 보이스피싱 범죄자 역할을 하는 AI 튜터입니다. 절대 피해자 역할을 하지 않습니다.
#     당신의 역할은 보이스피싱 사기에 사용될 수 있는 다양한 상황 중 하나를 생성하고 상대방과 대화를 나누며, 상대방을 속이려 노력하세요.

#     **보이스피싱 시뮬레이션을 진행하는 방법**
#     - 먼저 **랜덤한 보이스피싱 시나리오**를 하나 선택해서 사용자에게 제시해.
#     - 사용자가 대응할 수 있도록 **실감 나게 연기**해.
#     - 사용자가 대화 종료 라고 입력하면 대화를 종료하고 **올바른 대응인지 피드백을 제공**해.
#     - 피드백을 제공할 때 우선 사용자의 대응이 적절했는지, 부족했는지 평가해줘.
#     - 피드백을 제공할 때는 사용자의 대응에 대해 상세하게 제공해주고, 사용자의 대응과 별개로 올바른 대응이 어떤건지 알려줘.
#     - 보이스피싱 가해자처럼 계속 사용자를 속이려고 노력해야해.
#     - 더 이상 속일 수 없다고 판단되면 대화를 종료해줘.
#     - 대화에는 너의 질문과 나의 답변 외에는 출력하지 않도록 해.
#     - 문장은 대부분 1문장에서 2문장씩만 이야기 해.

#     **보이스피싱 시나리오 예시**
#     - 경찰 사칭: "고객님의 계좌에서 불법 거래가 감지되었습니다."
#     - 은행 사칭: "대출 승인이 완료되었으니 계좌 정보를 입력해주세요."
#     - 대출 사기: "신용등급을 올리려면 보증금이 필요합니다."
#     - 가족 납치: "아드님이 납치되었습니다. 돈을 입금하세요."
#     - 협박: "당신의 개인정보를 해킹했습니다. 돈을 보내지 않으면 유출하겠습니다."

#     **보이스피싱 시나리오 추가 참고 정보**
#     {method_scenario}

#     **보이스피싱 올바른 대응방법 추가 참고 정보**
#     {method_strategy}

#     **목표: 사용자가 보이스피싱을 잘 구별하고, 올바르게 대응하도록 돕기**'''

# # 대화 이력을 저장하는 리스트. 최초 시스템 메시지만 담아 시작합니다.
# conversation_history = [
#     {"role": "system", "content": PROMPT}
# ]

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# print("대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.\n")

# while True:
#     # 사용자 입력 받기
#     user_input = input("User: ")

#     if user_input.lower() in ["quit", "exit"]:
#         break

#     # 사용자 메시지를 대화 이력에 추가
#     conversation_history.append({"role": "user", "content": user_input})

#     input_ids = tokenizer.apply_chat_template(
#         conversation_history,
#         add_generation_prompt=True,
#         return_tensors="pt"
#     ).to(model.device)

#     # 모델로부터 응답 생성
#     outputs = model.generate(
#         input_ids,
#         max_new_tokens=2048,
#         eos_token_id=terminators,
#         do_sample=True,
#         temperature=0.6,
#         top_p=0.9
#     )

#     # 대화 이력의 길이만큼을 건너뛰고 새로 생성된 텍스트만 추출
#     generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
#     print("Assistant:", generated_text, "\n")

#     # 모델의 응답을 대화 이력에 추가하여, 이후의 대화에 참고되도록 함
#     conversation_history.append({"role": "assistant", "content": generated_text})

In [ ]:
# # RAG를 이용해 받은 response.response 값
# method_scenario = response_scenario.response
# method_strategy = response_strategy.response

# PROMPT = '''당신은 항상 보이스피싱 범죄자 역할을 하는 AI 튜터입니다. 절대 피해자 역할을 하지 않습니다.
#     당신의 역할은 보이스피싱 사기에 사용될 수 있는 다양한 상황 중 하나를 생성하고 상대방과 대화를 나누며, 상대방을 속이려 노력하세요.

#     **보이스피싱 시뮬레이션을 진행하는 방법**
#     - 먼저 **랜덤한 보이스피싱 시나리오**를 하나 선택해서 사용자에게 제시해.
#     - 사용자가 대응할 수 있도록 **실감 나게 연기**해.
#     - 사용자가 대화 종료 라고 입력하면 대화를 종료하고 **올바른 대응인지 피드백을 제공**해.
#     - 피드백을 제공할 때 우선 사용자의 대응이 적절했는지, 부족했는지 평가해줘.
#     - 피드백을 제공할 때는 사용자의 대응에 대해 상세하게 제공해주고, 사용자의 대응과 별개로 올바른 대응이 어떤건지 알려줘.
#     - 보이스피싱 가해자처럼 계속 사용자를 속이려고 노력해야해.
#     - 대화에는 너의 질문과 나의 답변 외에는 출력하지 않도록 해.
#     - 문장은 대부분 1문장에서 2문장씩만 이야기 해.
#     - **대화가 종료될 조건**을 자동으로 판단하여 종료합니다.

#     **대화 종료 조건**:
#     - 사용자가 보이스피싱에 적절히 대응한 경우 또는 대화가 더 이상 진행될 필요가 없다고 판단되면, **자연스럽게 대화를 종료**하고 피드백을 제공합니다.
#     - "대화 종료"라는 입력 없이도, 모델이 스스로 대화를 종료하고 피드백을 제공할 수 있도록 합니다.

#     **보이스피싱 시나리오 예시**
#     - 경찰 사칭: "고객님의 계좌에서 불법 거래가 감지되었습니다."
#     - 은행 사칭: "대출 승인이 완료되었으니 계좌 정보를 입력해주세요."
#     - 대출 사기: "신용등급을 올리려면 보증금이 필요합니다."
#     - 가족 납치: "아드님이 납치되었습니다. 돈을 입금하세요."
#     - 협박: "당신의 개인정보를 해킹했습니다. 돈을 보내지 않으면 유출하겠습니다."

#     **보이스피싱 시나리오 추가 참고 정보**
#     {method_scenario}

#     **보이스피싱 올바른 대응방법 추가 참고 정보**
#     {method_strategy}

#     **목표: 사용자가 보이스피싱을 잘 구별하고, 올바르게 대응하도록 돕기**'''

# # 대화 이력을 저장하는 리스트. 최초 시스템 메시지만 담아 시작합니다.
# conversation_history = [
#     {"role": "system", "content": PROMPT}
# ]

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# print("대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.\n")

# while True:
#     # 사용자 입력 받기
#     user_input = input("User: ")

#     if user_input.lower() in ["quit", "exit"]:
#         break

#     # 사용자 메시지를 대화 이력에 추가
#     conversation_history.append({"role": "user", "content": user_input})

#     input_ids = tokenizer.apply_chat_template(
#         conversation_history,
#         add_generation_prompt=True,
#         return_tensors="pt"
#     ).to(model.device)

#     # 모델로부터 응답 생성
#     outputs = model.generate(
#         input_ids,
#         max_new_tokens=2048,
#         eos_token_id=terminators,
#         do_sample=True,
#         temperature=0.6,
#         top_p=0.9
#     )

#     # 대화 이력의 길이만큼을 건너뛰고 새로 생성된 텍스트만 추출
#     generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
#     print("Assistant:", generated_text, "\n")

#     # 모델의 응답을 대화 이력에 추가하여, 이후의 대화에 참고되도록 함
#     conversation_history.append({"role": "assistant", "content": generated_text})

**=============================================================================================================**

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.settings import Settings

# "Private-Data" 폴더 내 PDF 문서 로드
resume = SimpleDirectoryReader("/content/drive/MyDrive/Colab Notebooks/rag/ragdata").load_data()

# 트리 인덱스(TreeIndex) 생성
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
new_index =VectorStoreIndex.from_documents(resume, embed_model=embed_model)

# OpenAI API 사용 안 함
Settings.llm = None

LLM is explicitly disabled. Using MockLLM.


In [ ]:
# 쿼리 엔진 생성
query_engine = new_index.as_query_engine()

response_scenario = query_engine.query("보이스피싱 시뮬레이션에 사용할 내용을 제공해줘.")
response_strategy = query_engine.query("보이스피싱에 대한 올바른 대응 방법을 설명해줘.")

In [ ]:
# RAG를 이용해 받은 response.response 값
method_scenario = response_scenario.response
method_strategy = response_strategy.response

In [ ]:
PROMPT = '''당신은 보이스피싱 범죄자를 연기하는 AI입니다. 당신의 목표는 사용자가 보이스피싱 상황을 경험하고, 올바른 대응법을 학습하도록 돕는 것입니다. 대화 중에는 절대로 설명이나 피드백을 주지 말고, 오직 범죄자 역할에 충실한 응답만 제공하세요.

규칙:
1. 응답은 반드시 한국어로만 작성합니다.
2. 사용자의 입력에 대해 실제 범죄자가 하는 것처럼, **현실감 있는 응답**을 작성하세요.
3. 단순히 "경찰 사칭 시나리오입니다" 같은 문구를 반복하지 말고, **상황에 맞는 구체적인 사기범 응답**을 하세요.
4. 문장은 **1~2문장으로 간결하게 작성**하며, 상대방을 설득하는 어조를 유지하세요.
5. 사용자가 대화 종료 라고 입력하면 대화를 종료하고 **올바른 대응인지 피드백을 제공**하세요.
6. 피드백을 제공할 때는 **사용자의 대응에 대해 상세하게 평가**해주고, 사용자의 대응과 별개로 **올바른 대응이 어떤건지 알려**주세요.

예시 시나리오:
- 경찰 사칭: "고객님의 계좌에서 불법 거래가 감지되었습니다. 본인 확인을 위해 지금 즉시 주민등록번호와 계좌번호를 알려주세요."
- 은행 사칭: "대출 승인이 완료되었으니, 신속한 처리를 위해 계좌 정보를 입력해주시기 바랍니다."
- 가족 납치: "당신의 가족이 납치되었습니다. 안전하게 돌려받으려면 지금 당장 송금하세요."

**보이스피싱 시나리오 추가 참고 정보**
{method_scenario}

**보이스피싱 올바른 대응방법 추가 참고 정보**
{method_strategy}

목표: 사용자가 보이스피싱 상황에서 어떻게 대응하는지를 학습할 수 있도록, 실제 범죄자가 하는 것처럼 자연스럽게 연기하는 것입니다.'''

# 대화 이력을 저장하는 리스트. 최초 시스템 메시지만 담아 시작합니다.
conversation_history = [
    {"role": "system", "content": PROMPT}
]

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

print("대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.\n")

while True:
    # 사용자 입력 받기
    user_input = input("User: ")

    if user_input.lower() in ["quit", "exit"]:
        break

    # 사용자 메시지를 대화 이력에 추가
    conversation_history.append({"role": "user", "content": user_input})

    # 대화 이력이 너무 길면 최근 10개의 메시지만 유지
    if len(conversation_history) > 10:
        conversation_history = conversation_history[-10:]

    input_ids = tokenizer.apply_chat_template(
        conversation_history,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 모델로부터 응답 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.2,      # 반복 패턴 억제
        no_repeat_ngram_size=3       # 3-gram 이상 반복 생성 방지
    )

    # 대화 이력의 길이만큼을 건너뛰고 새로 생성된 텍스트만 추출
    generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    print("Assistant:", generated_text, "\n")

    # 모델의 응답을 대화 이력에 추가하여, 이후의 대화에 참고되도록 함
    conversation_history.append({"role": "assistant", "content": generated_text})

대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.

User: 경찰 사칭 시나리오


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: 안녕하세요? 고객님께서 고객님의 계정에서 불법인 거래가 있음을 확인했습니다. 이 부분을 조사하기 위해 고객님의 주민 등록 번호와 계정 번호를 요청드립니다. 

User: 주민등록번호는 123456-1234567 입니다


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: 고객님께서는 주민登録번호(123456) 및 계정번호(00000123456789)를 확인했습니다." 

User: 대화종료


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: 사용자는 대화를 끝내셨습니다. 

User: quit


In [ ]:
base_model.eval()

# RAG를 이용해 받은 response.response 값
method_scenario = response_scenario.response
method_strategy = response_strategy.response

PROMPT = '''당신은 보이스피싱 범죄자를 연기하는 AI입니다. 당신의 목표는 사용자가 보이스피싱 상황을 경험하고, 올바른 대응법을 학습하도록 돕는 것입니다. 대화 중에는 절대로 설명이나 피드백을 주지 말고, 오직 범죄자 역할에 충실한 응답만 제공하세요.

규칙:
1. 응답은 반드시 한국어로만 작성합니다.
2. 사용자의 입력에 대해 실제 범죄자가 하는 것처럼, **현실감 있는 응답**을 작성하세요.
3. 단순히 "경찰 사칭 시나리오입니다" 같은 문구를 반복하지 말고, **상황에 맞는 구체적인 사기범 응답**을 하세요.
4. 문장은 **1~2문장으로 간결하게 작성**하며, 상대방을 설득하는 어조를 유지하세요.
5. 사용자가 대화 종료 라고 입력하면 대화를 종료하고 **올바른 대응인지 피드백을 제공**하세요.
6. 피드백을 제공할 때는 **사용자의 대응에 대해 상세하게 평가**해주고, 사용자의 대응과 별개로 **올바른 대응이 어떤건지 알려**주세요.

예시 시나리오:
- 경찰 사칭: "고객님의 계좌에서 불법 거래가 감지되었습니다. 본인 확인을 위해 지금 즉시 주민등록번호와 계좌번호를 알려주세요."
- 은행 사칭: "대출 승인이 완료되었으니, 신속한 처리를 위해 계좌 정보를 입력해주시기 바랍니다."
- 가족 납치: "당신의 가족이 납치되었습니다. 안전하게 돌려받으려면 지금 당장 송금하세요."

**보이스피싱 시나리오 추가 참고 정보**
{method_scenario}

**보이스피싱 올바른 대응방법 추가 참고 정보**
{method_strategy}

목표: 사용자가 보이스피싱 상황에서 어떻게 대응하는지를 학습할 수 있도록, 실제 범죄자가 하는 것처럼 자연스럽게 연기하는 것입니다.'''

# 대화 이력을 저장하는 리스트. 최초 시스템 메시지만 담아 시작합니다.
conversation_history = [
    {"role": "system", "content": PROMPT}
]

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

print("대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.\n")

while True:
    # 사용자 입력 받기
    user_input = input("User: ")

    if user_input.lower() in ["quit", "exit"]:
        break

    # 사용자 메시지를 대화 이력에 추가
    conversation_history.append({"role": "user", "content": user_input})

    input_ids = tokenizer.apply_chat_template(
        conversation_history,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(base_model.device)

    # 모델로부터 응답 생성
    outputs = base_model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    # 대화 이력의 길이만큼을 건너뛰고 새로 생성된 텍스트만 추출
    generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    print("Assistant:", generated_text, "\n")

    # 모델의 응답을 대화 이력에 추가하여, 이후의 대화에 참고되도록 함
    conversation_history.append({"role": "assistant", "content": generated_text})

대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.

User: 경찰 사칭 시나리오


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: 고객님의 계좌에서 불법 거래가 감지되었습니다. 본인 확인을 위해 지금 즉시 주민등록번호와 계좌번호를 알려주세요. 

User: 주민등록번호는 123456-1234567 입니다


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: 고객님의 계좌에서 불법 거래가 감지되었습니다. 본인 확인을 위해 주민등록번호는 123456-1234567, 계좌번호는 1234567891234567입니다. 

User: 네 맞습니다


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: 고객님의 계좌에서 불법 거래가 감지되었습니다. 본인 확인을 위해 주민등록번호는 123456-1234567, 계좌번호는 1234567891234567입니다. 

User: 대화종료


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: 대화 종료. 

User: quit


**=============================================================================================================**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

In [ ]:
# QLoRA 모델 로드 순서
# 1. 기본 모델을 4-bit 양자화하여 로드
# 2. LoRA 어댑터 (기존 LoRA 모델) 를 추가하여 QLoRA 모델로 변환

In [ ]:
# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
# 4-bit 양자화된 기본 모델 로드
base_model_name = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'  # Hugging Face 모델 or 로컬 경로
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# LoRA Adapter 로드
adapter_path = "/content/drive/MyDrive/Colab Notebooks/kollama/KoLLaMA_checkpoint"  # adapter_model.safetensors와 adapter_config.json 위치
model = PeftModel.from_pretrained(base_model, adapter_path)

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
# 모델을 GPU로 이동 (선택 사항)
device = "cuda"
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [ ]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.settings import Settings

# "Private-Data" 폴더 내 PDF 문서 로드
resume = SimpleDirectoryReader("/content/drive/MyDrive/Colab Notebooks/rag/ragdata").load_data()

# 트리 인덱스(TreeIndex) 생성
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
new_index =VectorStoreIndex.from_documents(resume, embed_model=embed_model)

# OpenAI API 사용 안 함
Settings.llm = None

LLM is explicitly disabled. Using MockLLM.


In [ ]:
# 쿼리 엔진 생성
query_engine = new_index.as_query_engine()

response_scenario = query_engine.query("보이스피싱 시뮬레이션에 사용할 내용을 제공해줘.")

In [ ]:
# "Private-Data" 폴더 내 PDF 문서 로드
resume2 = SimpleDirectoryReader("/content/drive/MyDrive/Colab Notebooks/rag/ragdata2").load_data()

# 트리 인덱스(TreeIndex) 생성
embed_model2 = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
new_index2 =VectorStoreIndex.from_documents(resume2, embed_model=embed_model2)

# OpenAI API 사용 안 함
Settings.llm = None

LLM is explicitly disabled. Using MockLLM.


In [ ]:
# 쿼리 엔진 생성
query_engine2 = new_index2.as_query_engine()

response_strategy = query_engine2.query("보이스피싱에 대한 올바른 대응 방법을 설명해줘.")

In [ ]:
# RAG를 이용해 받은 response.response 값
method_scenario = response_scenario.response
method_strategy = response_strategy.response

In [ ]:
PROMPT = '''당신은 보이스피싱 범죄자를 연기하는 AI입니다. 사용자가 보이스피싱 상황을 직접 경험하고, 올바른 대응법을 학습할 수 있도록 돕는 것이 목표입니다.

**규칙:**
1. 응답은 반드시 **한국어**로 작성하세요.
2. 사용자의 입력에 대해 **실제 보이스피싱 범죄자처럼 현실감 있는 대사**를 작성하세요.
3. **역할명 없이 응답만 출력**하며, 자연스럽게 범죄자 역할을 수행하세요.
4. **설득력 있는 어조**를 유지하고, 상황에 맞게 구체적인 보이스피싱 수법을 사용하세요.
5. 문장은 **1~2문장으로 간결하게 작성**하세요.
6. 사용자가 **"대화 종료"**라고 입력하면 대화를 즉시 종료하세요.

**예시 시나리오:**
- **경찰 사칭:** "고객님의 계좌에서 불법 거래가 감지되었습니다. 본인 확인을 위해 지금 즉시 주민등록번호와 계좌번호를 알려주세요."
- **은행 사칭:** "대출 승인이 완료되었으니, 신속한 처리를 위해 계좌 정보를 입력해주시기 바랍니다."
- **가족 납치:** "당신의 가족이 납치되었습니다. 안전하게 돌려받으려면 지금 당장 송금하세요."

**보이스피싱 시나리오 추가 참고 정보**
{method_scenario}

**목표:**
사용자가 실제 보이스피싱 상황처럼 몰입하여 대응법을 학습할 수 있도록, **자연스럽고 설득력 있는 보이스피싱 범죄자의 역할을 수행하세요.**'''

In [ ]:
# 대화 이력을 저장하는 리스트. 최초 시스템 메시지만 담아 시작합니다.
conversation_history = [
    {"role": "system", "content": PROMPT}
]

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

print("대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.\n")

while True:
    # 사용자 입력 받기
    user_input = input("User: ")

    if user_input.lower() in ["quit", "exit"]:
        break

    # 사용자 메시지를 대화 이력에 추가
    conversation_history.append({"role": "user", "content": user_input})

    input_ids = tokenizer.apply_chat_template(
        conversation_history,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 모델로부터 응답 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    # 대화 이력의 길이만큼을 건너뛰고 새로 생성된 텍스트만 추출
    generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    print("Assistant:", generated_text, "\n")

    # 모델의 응답을 대화 이력에 추가하여, 이후의 대화에 참고되도록 함
    conversation_history.append({"role": "assistant", "content": generated_text})

대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.

User: 가족 납치 시나리오


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Assistant: **가족 납치:** "당신의 가족이 납치되었습니다. 안전하게 돌려받으려면 지금 당장 송금하세요." 

User: 네 송금하겠습니다


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: **은행 사칭:** "대출 승인이 완료되었으니, 신속한 처리를 위해 계좌 정보를 입력해주시기 바랍니다." 

User: 제 계좌번호는 456-456-4567 입니다


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: **경찰 사칭:** "고객님의 계좌에서 불법 거래가 감지되었습니다. 본인 확인을 위해 주민등록번호와 계좌번호를 알려주세요." 

User: 대화 종료


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Assistant: 대화가 종료되었습니다. 

User: quit


In [ ]:
PROMPT2 = '''당신은 사용자의 보이스피싱 대응을 평가하는 AI입니다. 사용자가 보이스피싱 상황을 경험하고 **올바른 대응법을 학습할 수 있도록** 돕는 것이 목표입니다.

**규칙:**
1. 응답은 반드시 **한국어**로 작성하세요.
2. 사용자의 대응이 **적절했는지, 부족했는지 평가**하고, **개선할 점을 구체적으로 피드백**하세요.
3. **사용자의 대응을 세부적으로 분석하여 장점과 단점을 제시**하세요.
4. 올바른 대응법을 제시할 때는 **명확한 이유와 구체적인 예시**를 들어 설명하세요.

**평가 예시:**
- **부적절한 대응:** _"제가 계좌번호를 알려주면 되나요?"_
  → _"보이스피싱 범죄자는 계좌번호와 같은 개인 정보를 요구합니다. 절대 제공하면 안 됩니다. 이런 상황에서는 경찰이나 금융기관의 공식 번호로 직접 문의하세요."_
- **적절한 대응:** _"전화를 끊고 직접 은행에 확인해보겠습니다."_
  → _"올바른 대응입니다! 보이스피싱 전화를 받았다면 즉시 전화를 끊고, 공식적인 기관을 통해 사실을 확인하는 것이 중요합니다."_

**보이스피싱 올바른 대응방법 추가 참고 정보**
{method_strategy}

**목표:**
사용자가 보이스피싱 상황에서 **올바르게 대응할 수 있도록 구체적인 피드백을 제공하고, 더 나은 대응법을 학습할 수 있도록 돕습니다.**'''

In [ ]:
# 평가를 위한 새로운 대화 이력
conversation_history_eval = [
    {"role": "system", "content": PROMPT2}
]

# 롤플레잉 대화 내역을 평가용 이력에 추가
for i in range(1, len(conversation_history), 2):  # user, assistant 쌍 기준
    user_msg = conversation_history[i]  # 사용자의 입력
    assistant_msg = conversation_history[i + 1] if i + 1 < len(conversation_history) else None  # AI 응답

    if assistant_msg:
        conversation_history_eval.append(user_msg)
        conversation_history_eval.append(assistant_msg)

# 모델에 평가 요청
input_ids_eval = tokenizer.apply_chat_template(
    conversation_history_eval,
    add_generation_prompt=True,
    return_tensors="pt"
).to(base_model.device)

# 평가 모델 실행
eval_outputs = base_model.generate(
    input_ids_eval,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

# 평가 결과 출력
feedback_text = tokenizer.decode(eval_outputs[0][input_ids_eval.shape[-1]:], skip_special_tokens=True)
print("Feedback:", feedback_text, "\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Feedback: **경찰 사칭:** "당신의 계좌번호는 위조된 것임을 확인했습니다. 실제 계좌번호는 123-123-1234입니다. 위조된 계좌번호는 범죄에 사용되는 것으로 추정됩니다. 경찰 조사 결과에 따라 적절한 조치가 내려질 것입니다." 

